# Core Model Guide: StVenantModel (1D Hydrodynamics)

This notebook provides a guide to the `StVenantModel`, the most advanced 1D hydraulic model in the CHS SDK. It is used for high-fidelity simulation of unsteady flow in networks of open channels.

## 1. Theoretical Background

This model solves the **Saint-Venant equations**, which are a set of nonlinear partial differential equations that describe the flow of water in an open channel. They are derived from the principles of conservation of mass and conservation of momentum.

Unlike simpler routing models (like the Muskingum model), the St. Venant model can simulate a wide range of dynamic effects, including:
- Backwater effects (downstream conditions affecting upstream flow).
- Wave propagation and reflection.
- Supercritical and subcritical flow regimes.

The CHS SDK implementation solves these equations for an entire **network** of channels (`Reaches`) and junctions (`Nodes`), making it a powerful tool for complex system simulation.

## 2. API and Parameters

The `StVenantModel` is a high-level wrapper. It is initialized by providing data that defines the network structure, rather than simple numerical parameters.

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Add the project root to the path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from water_system_sdk.src.chs_sdk.modules.modeling.st_venant_model import StVenantModel

The constructor takes three main arguments:

- `nodes_data` (List[Dict]): A list where each dictionary defines a node. A node can be a `junction`, an `inflow` boundary, or a `level` boundary.
- `reaches_data` (List[Dict]): A list where each dictionary defines a channel reach, including its geometry (length, slope, etc.) and which nodes it connects.
- `structures_data` (List[Dict], optional): A list defining any hydraulic structures like weirs or gates in the network.

## 3. Code Example: Two Reservoirs Connected by a Channel

We will create a simple network with two reservoirs (represented by `LevelBoundary` nodes) connected by a single channel (`Reach`). We will set one reservoir to a higher water level than the other and observe as the system dynamically seeks equilibrium.

In [ ]:
# 1. Define the Network Configuration
nodes = [
    # Upstream reservoir with a fixed high water level
    {"name": "n_up", "type": "level", "level": 10.0, "bed_elevation": 0.0},
    # Downstream reservoir with a fixed low water level
    {"name": "n_down", "type": "level", "level": 8.0, "bed_elevation": 0.0}
]

reaches = [
    {
        "name": "r1",
        "from_node": "n_up",
        "to_node": "n_down",
        "length": 1000.0, # 1km long channel
        "manning_coefficient": 0.03,
        "bottom_width": 10.0,
        "side_slope": 1.0, # Trapezoidal channel
        "bed_slope": 0.001,
        "discharge": 0.0 # Start with no flow
    }
]

# 2. Initialize the Model
model = StVenantModel(nodes_data=nodes, reaches_data=reaches)

# 3. Simulation Setup
dt = 10.0 # 10-second time step
n_steps = 200
history = []

# 4. Simulation Loop
print("Starting simulation...")
for t in range(n_steps):
    # Run one step of the solver
    model.step(dt=dt)
    
    # Record the state
    state = model.get_state()
    state['time'] = t * dt
    history.append(state)

print("Simulation complete.")

## 4. Visualization

We can now process the history and plot the discharge in the connecting channel over time.

In [ ]:
# Extract the discharge history for reach 'r1'
time_series = [s['time'] for s in history]
discharge_series = [s['reaches']['r1']['discharge'] for s in history]

plt.figure(figsize=(12, 6))
plt.plot(time_series, discharge_series, label='Discharge in Channel r1')

plt.title('Dynamic Flow Between Two Reservoirs')
plt.xlabel('Time (s)')
plt.ylabel('Discharge (m³/s)')
plt.legend()
plt.grid(True)
plt.show()

### Analysis of Results

The plot shows that initially, there is a large flow from the upstream reservoir to the downstream one due to the large head difference. As the simulation progresses, the system stabilizes, and the flow rate settles at a constant value, representing the steady-state flow for this particular channel geometry and boundary conditions. 

This example demonstrates how the `StVenantModel` can be used to simulate the dynamic hydraulic interactions in a network.